In [1]:
#r "C:\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
using BoSSS.Application.IBMSolver;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Solution;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using ilPSP.Tracing;

In [1]:
string WFlowName = "SolverPerformanceDrivenCavity";
WorkflowMgm.Init(WFlowName);

In [1]:
var WinTempDB = OpenOrCreateDatabase(@"\\dc1\userspace\weber\Performance_DB");
//var LinuxTempDB = OpenOrCreateDatabase(@"L:\BoSSS_DB\Lichtenberg_DB");

In [1]:
//Console.WriteLine("Please type in your Username");
//var Username = Console.ReadLine();

In [1]:
//var myBatch = new SlurmClient(@"L:\SLURM","lcluster3.hrz.tu-darmstadt.de","oe11okuz");
var myBatch = new MsHPC2012Client(@"\\dc1\userspace\weber\deploy","hpccluster",ComputeNodes : new[] {"hpccluster"});

In [1]:
//var myBatch = new MiniBatchProcessorClient(@"P:\tmp\");

In [1]:
//MiniBatchProcessor.Server.StartIfNotRunning();

In [1]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }
}

In [1]:
// ==========
// Parameters
// ==========

In [1]:
LinearSolverCodes[] linSolvers = new LinearSolverCodes[] {
    //LinearSolverCodes.exp_schwarz_directcoarse,
    //LinearSolverCodes.exp_schwarz_Kcycle_directcoarse,
    //LinearSolverCodes.exp_schwarz_directcoarse_overlap,
    //LinearSolverCodes.exp_schwarz_Kcycle_directcoarse_overlap,
    //LinearSolverCodes.exp_softgmres_schwarz_Kcycle_directcoarse_overlap,
    //LinearSolverCodes.exp_softgmres_schwarz_directcoarse_overlap,
    //LinearSolverCodes.automatic,
    //LinearSolverCodes.classic_mumps,
    //LinearSolverCodes.exp_multigrid
};
NonlinearSolverCodes[] NlinSolvers = new NonlinearSolverCodes[]{
    NonlinearSolverCodes.NewtonGMRES,
    NonlinearSolverCodes.Newton,
    NonlinearSolverCodes.PicardGMRES,
    NonlinearSolverCodes.Picard
};
int[] ks = new int[] {2,3};
int re        = 400; 
int saveToDB  = 1;
int[] procs    = new int[] {1};
//string[] estTime = new string[] {"24:00:00","18:00:00","12:00:00","06:00:00"};

In [1]:
// ===============================
// Init grids and save to database
// ===============================

In [1]:
//int[] Resolutions_3D = new int[] {5,7,10,13};//,17,25};
int[] Resolutions_3D = new int[] { 2, 4, 8, 16, 32, 64 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
  int  Res = Resolutions_3D[cnt];
// x-direction
var _xNodes = GenericBlas.Linspace(-0.5, 0.5, Res + 1);
// y-direction
var _yNodes = GenericBlas.Linspace(-0.5, 0.5, Res + 1);
// z-direction
var _zNodes = GenericBlas.Linspace(-0.5, 0.5, Res + 1);
// Generate Grid
int J           = (_xNodes.Length - 1)*(_yNodes.Length - 1)*(_zNodes.Length - 1);
string GridName = string.Format(WorkflowMgm.CurrentProject + "_J" + J);
grids[cnt] = WinTempDB.Grids.SingleOrDefault(grd => grd.Name == GridName);
if(grids[cnt] == null){
Console.WriteLine("Creating grid with " + J + " cells.");
GridCommons grd; 
grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes, false, true, false, CellType.Cube_Linear);
grd.EdgeTagNames.Add(1, "Velocity_17inlet");
grd.EdgeTagNames.Add(2, "Wall");
grd.DefineEdgeTags(delegate (double[] _X) {
                    var X    = _X;
                    double x = X[0];
                    double y = X[1];
                    double z = X[2];
                    byte ret = 0;

                    if (Math.Abs(x - (-0.5)) < 1.0e-6)
                        // inlet
                        ret = 2;

                    if (Math.Abs(x - (0.5)) < 1.0e-6)
                        // outlet
                        ret = 2;

                    if (Math.Abs(y - (-0.5)) < 1.0e-6)
                        // left
                        ret = 2;

                    if (Math.Abs(y - (0.5)) < 1.0e-6)
                        // right
                        ret = 2;

                    if (Math.Abs(z - (-0.5)) < 1.0e-6)
                        // top left
                        ret = 2;

                    if (Math.Abs(z - (0.5)) < 1.0e-6)
                        // top right
                        ret = 1;
                        
                        return ret;
                });
WinTempDB.SaveGrid(ref grd);
grids[cnt] = grd;
}else{
Console.WriteLine("Found Grid: " + grids[cnt]);
if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
}
}

In [1]:
// =====================================
// setup control object for a solver run
// =====================================

In [1]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [1]:
static class InitialValues {
    public static double ZERO(double[] X) {
        return 0.0;
    }
    public static double Phi(double[] X) {
        return -1;   
    }
    public static double VelocityInlet(double[] X) {
        return 1;
    }
}

In [1]:
List<IBM_Control> controls = new List<IBM_Control>();

In [1]:
// polynomial degrees to test
int[] PolyDegS = new int[] {2};

In [1]:
// MaxDof for one claculation (we skip fine grids for higher polynomial orders)
int MaxDof = 5000000;

In [1]:
controls.Clear();
foreach(LinearSolverCodes linSolverName in linSolvers){
foreach(int k in PolyDegS) {
int iGrd = 0;    
foreach(IGridInfo grd in grids) {
    iGrd ++;
    var C = new IBM_Control();
    controls.Add(C);

    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > MaxDof)
        continue;
    
	// ===!!!=== CAUTION ===!!!===
    // Solverframework (SolverEnum numbers) have changed! Please check LinearSolver and NonLinearSolver for details
	
    string caseName = string.Format("J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);

// basic database options
// ======================

C.savetodb = true;
C.SetDatabase(WinTempDB);
C.DynamicLoadBalancing_Period                = 1;
C.GridPartType                               = GridPartType.Hilbert;
C.DynamicLoadBalancing_RedistributeAtStartup = false;
C.DynamicLoadBalancing_ImbalanceThreshold    = 0.0;

C.saveperiod  = 1;
C.ProjectName = "3DChannel";
C.Tags.Add("Prec param study");
C.solver_name = linSolverName;
C.SetDGdegree(k);
C.SetGrid(grd);
           
// Set Initial Conditions
C.InitialValues.Add("VelocityX", GetFormulaObject(InitialValues.ZERO));
C.InitialValues.Add("VelocityY", GetFormulaObject(InitialValues.ZERO));
C.InitialValues.Add("VelocityZ", GetFormulaObject(InitialValues.ZERO));
C.InitialValues.Add("Pressure", GetFormulaObject(InitialValues.ZERO));
C.InitialValues.Add("Phi", GetFormulaObject(InitialValues.Phi));
        

        // Because its a sphere
  

        // Physical values
        C.PhysicalParameters.rho_A = 1;
        // 1/Re
        //C.PhysicalParameters.mu_A = 1.0 / 10.0;
        C.PhysicalParameters.mu_A = 1.0/ re;

        //C.PhysicalParameters.mu_A = 1.0 / re;

        // Boundary conditions
        C.AddBoundaryValue("Velocity_inlet", "VelocityX", GetFormulaObject(InitialValues.VelocityInlet));
        C.AddBoundaryValue("Velocity_inlet", "VelocityY", GetFormulaObject(InitialValues.ZERO));
        C.AddBoundaryValue("Wall");


        // misc. solver options
        // ====================
        C.PhysicalParameters.IncludeConvection                     = true;
        C.AdvancedDiscretizationOptions.PenaltySafety              = 4;
        C.AdvancedDiscretizationOptions.CellAgglomerationThreshold = 0.2;
        C.LevelSetSmoothing                                        = false;
        C.MaxKrylovDim                                             = 30;
        C.MaxSolverIterations                                      = 50;
        C.MinSolverIterations                                      = 1;
        // C.MaxSolverIterations = 10000;
        C.Solver_ConvergenceCriterion = 1E-5;
        //C.Solver_ConvergenceCriterion = 1E-6;
        C.VelocityBlockPrecondMode = MultigridOperator.Mode.SymPart_DiagBlockEquilib_DropIndefinite;
        
        // Solver configuration
        C.NonlinearSolve = NonlinearSolverCodes.NewtonGMRES;
        C.LinearSolve    = linSolverName;

        

        // Timestepping
        // ============
        C.Timestepper_Scheme = IBM_Control.TimesteppingScheme.BDF2;
        //C.dtFixed            = 1E20;
        C.dtMax               = 1E20;
        C.dtMin               = 1E20;
        C.Endtime             = 10000000;
        C.NoOfTimesteps       = 1;
        C.NoOfMultigridLevels = 3;
        
        // Assign correct names
        C.SessionName = "DrivenCavity_k2_"+C.NonlinearSolve+"_"+linSolverName+"_J"+grd.NumberOfCells+"_MGLev"+C.NoOfMultigridLevels;
        }
}
}

In [1]:
LastError

In [1]:
// ==========
// Launch Job
// ==========

In [1]:
foreach(var ctrl in controls) {
    var tmpName = ctrl.SessionName;
foreach(int proc in procs){
    ctrl.SessionName = tmpName;
    Console.WriteLine(" Submitting: " + ctrl.SessionName);
    ctrl.RunBatch(myBatch,NumberOfMPIProcs:proc,UseComputeNodesExclusive:true,executionTime:"96:00:00");  
}}

In [1]:
WorkflowMgm.SessionTable.Rows.Count;

In [1]:
// =====================================
// Session table
// =====================================

In [1]:
// -------------------------------
// Additional/user-defined columns
// -------------------------------

In [1]:
static class AddCols {
     static public double getMaxTime(ISessionInfo SI, string method, bool exclusive) {
         double maxTime = 0.0;
         var mcr        = SI.GetProfiling();
                // Iterate over MPIs
                for (int j = 0; j < mcr.Count(); j++) {
                    MethodCallRecord value;
                    // Iterate over methods
                        // Get execution time of current method for current processor
                        double tempTime;
                        double tempFractions;
                        int occurence = 1;

                        value = mcr[j].FindChild(method);
                        if (value == null) {
                            value = mcr[j];
                        }
                        if (exclusive) {
                            tempTime = value.FindChildren(method).OrderByDescending(s => s.TimeExclusive.TotalSeconds).Pick(occurence-1).TimeExclusive.TotalSeconds;
                                IEnumerable<MethodCallRecord> calls = value.FindChildren(method).OrderByDescending(s => s.ExclusiveTimeFractionOfRoot);
                                double maxValue = calls.Pick(occurence-1).ExclusiveTimeFractionOfRoot;
                                int maxIndex    = calls.Select(s => s.ExclusiveTimeFractionOfRoot).ToList().IndexOf(maxValue);
                                tempFractions   = maxValue;                             
                            
                        } else {
                             tempTime = value.FindChildren(method).OrderByDescending(s => s.TimeSpentInMethod.TotalSeconds).Pick(occurence-1).TimeSpentInMethod.TotalSeconds;
                                IEnumerable<MethodCallRecord> calls = value.FindChildren(method).OrderByDescending(s => s.TimeFractionOfRoot);
                                double maxValue = calls.Pick(occurence-1).TimeFractionOfRoot;
                                int maxIndex    = calls.Select(s => s.TimeFractionOfRoot).ToList().IndexOf(maxValue);
                                tempFractions   = maxValue;
                        }
                            
                        // Only save execution time if it is the highest value of all processor times
                        if (tempTime > maxTime) {
                            maxTime = tempTime;
                        }
                    }
                    return maxTime;
    }
    
    static public object SlvIterTime(ISessionInfo SI) {
       // Thread.Sleep(2000);
        return getMaxTime(SI,"Slv Iter",false);
    }
    static public object SlvInitTime(ISessionInfo SI) {
        // sometimes problems such that some method show NULL time. Sleep helped
        //Thread.Sleep(2000);
        return getMaxTime(SI,"Slv Init",false);
    }
    //static public object SchwarzSolve(ISessionInfo SI) {
      //  Thread.Sleep(2000);
        //return getMaxTime(SI,"BoSSS.Solution.Multigrid.Schwarz.Solve",false);
    //}
    //static public object NewtonDirder(ISessionInfo SI) {
      //  Thread.Sleep(2000);        
        //return getMaxTime(SI,"BoSSS.Solution.Multigrid.Newton.dirder",false);
    //}
    static public object MPIs(ISessionInfo SI) {
        return SI.ComputeNodeNames.Count();
    }
}

In [1]:
WorkflowMgm.AdditionalSessionTableColums.Clear();
WorkflowMgm.AdditionalSessionTableColums.Add(
    "SlvIter_excl", AddCols.SlvIterTime);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Processors", AddCols.MPIs);

In [1]:
// --------------------------
// Save Session table in file
// --------------------------

In [1]:
var SessTab = WorkflowMgm.SessionTable;

In [1]:
// The Session column can't be serialized,
// we have to remove it
List<string> AllCols = SessTab.GetColumnNames().ToList();
AllCols.Remove("Session");

In [1]:
SessTab = SessTab.ExtractColumns(AllCols.ToArray());

In [1]:
AllCols

In [1]:
//Checking if table contains all information
var SubTab_1 = SessTab.ExtractColumns("Grid:NoOfCells","SlvIter_excl","NonlinearSolve","LinearSolve","NoOfMultigridLevels","DGdegree:Velocity*","RegularTerminated");

In [1]:
SubTab_1.Print();

In [1]:
using System.IO;

In [1]:
// Filename
var now           = DateTime.Now;
SessTab.TableName = "SolverRuns--" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath    = Path.Combine(CurrentDocDir, SessTab.TableName + ".json");

In [1]:
// save session table as file: this can be stored in our git 
// repository
SessTab.SaveToFile(docpath);